In [37]:
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from pandas import concat
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.model_selection import cross_val_score
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model
from sklearn import preprocessing
from sklearn.preprocessing import normalize, scale
from sklearn.model_selection import KFold
from tensorflow.compat.v1.losses import Reduction
from google.colab import files
from tensorflow.keras.layers import Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
data = open("/content/drive/MyDrive/dataset/dataset/data/x_train.csv")
x_train = np.loadtxt(data, delimiter=",")
#print(x_train)

data = open("/content/drive/MyDrive/dataset/dataset/data/x_test.csv")
x_test = np.loadtxt(data, delimiter=",")
#print(x_test)

data = open("/content/drive/MyDrive/dataset/dataset/data/train-label.dat")
y_train = np.loadtxt(data, delimiter=" ", dtype='int')
#print(y_train)

data = open("/content/drive/MyDrive/dataset/dataset/data/test-label.dat")
y_test = np.loadtxt(data, delimiter=" ", dtype='int')
#print(y_test)

vocab = pd.read_csv('/content/drive/MyDrive/dataset/dataset/data/vocabs.txt', header=None)

In [ ]:
# Scaling the training and testing data
# Performing centering
x_train = x_train - x_train.mean()
print(x_train)
y_train = y_train - y_train.mean()
print(y_train)
# Performing Normalization
# scaler = Normalizer()
# x_train = scaler.fit_transform(x_train)
# print(x_train)
# y_train = scaler.fit_transform(y_train)
# print(y_train)
# Performing Standardization
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# print(x_train)
# y_train = scaler.fit_transform(y_train)
# print(y_train)

In [ ]:
epoch = 20
splits = 5
kfold = KFold(n_splits=splits, shuffle=True, random_state=2)

loss = []
val_loss = []
accuracy = []
val_accuracy = []

for i, (train, test) in enumerate(kfold.split(x_train)):
  model = tf.keras.models.Sequential()
  #model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Flatten(input_shape=(8520,1)))
  #model.add(tf.keras.layers.Dense(units=20, activation=tf.nn.relu))
  model.add(tf.keras.layers.Dense(units=4270, activation=tf.nn.relu))
  model.add(Dropout(0.9))
  #model.add(tf.keras.layers.Dense(units=2135, activation=tf.nn.relu))
  #model.add(tf.keras.layers.Dense(units=4270, activation=tf.nn.relu))
  #model.add(tf.keras.layers.Dense(units=5124, activation=tf.nn.relu))
  #model.add(tf.keras.layers.Dense(units=8540, activation=tf.nn.relu))
  model.add(tf.keras.layers.Dense(units=20, activation=tf.nn.sigmoid))
  #model.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy'])
  #model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])
  model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.6), loss='mean_squared_error', metrics=['accuracy'])
  #model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.6), loss='binary_crossentropy', metrics=['accuracy'])

  # x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=30, train_size=0.2)

  # callback = [EarlyStopping(monitor='val_accuracy', patience=5)]
  history = model.fit(x_train, y_train, epochs=epoch, validation_data=(x_test, y_test), batch_size=32)
  los, acc = model.evaluate(x_test, y_test)
  loss.append(history.history['loss'])
  val_loss.append(history.history['val_loss'])
  accuracy.append(history.history['accuracy'])
  val_accuracy.append(history.history['val_accuracy'])

  print(loss)
  print(val_loss)
  print(acc)
  print(val_accuracy)

# Creating the mean values for the cross validation
mean_loss = []
mean_val_loss = []
mean_accuracy = []
mean_val_accuracy = []
j=0
for i in range(len(loss[j])):
    temp_loss = 0
    temp_val_loss = 0
    temp_accuracy = 0
    temp_val_accuracy = 0
    for j in range(splits):
        temp_loss = temp_loss + loss[j][i]
        temp_val_loss = temp_val_loss + val_loss[j][i]
        temp_accuracy = temp_accuracy + accuracy[j][i]
        temp_val_accuracy = temp_val_accuracy + val_accuracy[j][i]
    mean_loss.append(temp_loss/ splits)
    mean_val_loss.append(temp_val_loss / splits)
    mean_accuracy.append(temp_accuracy / splits)
    mean_val_accuracy.append(temp_val_accuracy / splits)

print(mean_loss)
print(mean_val_loss)
print(mean_accuracy)
print(mean_val_accuracy)
# Plot training and
# loss = history.history['loss']
# val_loss = history.history['val_loss']
epochs = range(1, len(mean_loss) + 1)
plt.plot(epochs, mean_loss, 'y', label='Training loss')
plt.plot(epochs, mean_val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(["Train", 'Validation'], loc='upper right')
plt.savefig('/content/drive/MyDrive/plots/loss.png')
plt.savefig("loss.png")
files.download("loss.png") 
plt.show()


# Plot training and validation accuracy values
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
plt.plot(epochs, mean_accuracy, 'y', label='Training acc')
plt.plot(epochs, mean_val_accuracy, 'r', label='Validation acc')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.savefig('/content/drive/MyDrive/plots/acc.png')
plt.savefig("acc.png")
files.download("acc.png") 
plt.show()

print(los)
print(acc)